In [ ]:
!pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d43724f1c45e7c20bd42445212bf1a9fc57f214f3b4cdc169530a31d54269c00
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import logging
import os
import random
import re
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset, Dataset
from sklearn.metrics import f1_score
import numpy as np
from torch import nn
import glob
import shutil
from transformers import AutoModelForMaskedLM , AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
import transformers
from transformers import (
    Trainer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback,

)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import numpy as np
from torch import nn
from transformers.file_utils import ModelOutput
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification , AutoModelForTokenClassification
import pandas as pd
from ast import literal_eval
from scipy.special import expit
import torch
from transformers import AutoModelForQuestionAnswering
import evaluate
from tqdm import tqdm
import collections
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import DataCollatorForTokenClassification
from sklearn.metrics import precision_recall_fscore_support, classification_report
from seqeval.metrics import classification_report

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
batch_size = 4
lr = 2e-5
EPOCHS = 2

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
train_dataset_  = load_dataset("gtfintechlab/finer-ord-bio",split = "train[:10]",trust_remote_code=True)
val_dataset_  = load_dataset("gtfintechlab/finer-ord-bio",split = "validation[:10]",trust_remote_code=True)
test_dataset_  = load_dataset("gtfintechlab/finer-ord-bio",split = "test[:10]",trust_remote_code=True)

README.md:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train-00000-of-00001.parquet:   0%|          | 0.00/279k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3262 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/402 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1075 [00:00<?, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
train_dataset_[0]["tokens"]

['Kenyan',
 'Firms',
 'Eye',
 'Deals',
 'During',
 'Obama',
 'Summit',
 'Tagged',
 ':',
 'The',
 'Global',
 'Entrepreneurship',
 'Summit',
 ',',
 'launched',
 'by',
 'President',
 'Obama',
 'in',
 '2009',
 ',',
 'brings',
 'together',
 'entrepreneurs',
 'and',
 'investors',
 'from',
 'across',
 'Africa',
 'and',
 'around',
 'the',
 'world',
 'annually',
 'to',
 'showcase',
 'innovative',
 'projects',
 ',',
 'exchange',
 'new',
 'ideas',
 ',',
 'and',
 'help',
 'spur',
 'economic',
 'opportunity',
 '.']

In [ ]:
train_dataset_

Dataset({
    features: ['tokens', 'tags'],
    num_rows: 10
})

In [ ]:
train_dataset_[0]['tags']

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
train_dataset = train_dataset_.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset_.column_names,
)

val_dataset = val_dataset_.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=val_dataset_.column_names,
)

test_dataset = test_dataset_.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=test_dataset_.column_names,
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
label_names = ['O', 'PER_B' , 'PER_I' , 'LOC_B' , 'LOC_I', 'ORG_B' , 'ORG_I' ]

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
#{'O': 0, 'PER_B': 1, 'PER_I': 2, 'LOC_B': 3, 'LOC_I': 4, 'ORG_B': 5, 'ORG_I': 6}

In [ ]:

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    id2label=id2label,
    label2id=label2id,
    num_hidden_layers = 1
).to(device)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    val_dataset, collate_fn=data_collator, batch_size=batch_size
)

test_dataloader = DataLoader(
    test_dataset, collate_fn=data_collator, batch_size=batch_size
)

In [ ]:
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.01)

steps_per_epoch = len(train_dataloader)

total_training_steps = steps_per_epoch * EPOCHS

warmup_steps = int(0.05 * total_training_steps)


scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

In [ ]:
num_train_epochs = 1

In [ ]:
def postprocess(predictions, labels):
    # predictions = predictions.detach().cpu().clone().numpy()
    # labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
all_predictions = []
all_labels = []

for batch in tqdm(eval_dataloader):
    # Move batch data to device

    batch = {key: value.to(device) for key, value in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    # Collect predictions and labels for metric calculation
    all_predictions.extend(predictions.cpu().detach().numpy())
    all_labels.extend(labels.cpu().detach().numpy())

    break

# Compute metrics
true_predictions, true_labels = postprocess(all_predictions, all_labels)
metric.add_batch(predictions=true_predictions, references=true_labels)
results = metric.compute(zero_division=0)


  0%|          | 0/3 [00:00<?, ?it/s]
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packag

In [ ]:
# from sklearn.metrics import precision_recall_fscore_support, classification_report
from seqeval.metrics import classification_report

In [ ]:
print(classification_report(true_predictions,true_labels,labels=label_names))

TypeError: classification_report() got an unexpected keyword argument 'labels'

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

In [ ]:
flat_predictions = [label for sublist in true_predictions for label in sublist]
flat_references = [label for sublist in true_labels for label in sublist]

In [ ]:
label_names = ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']

# Generate the classification report
report = classification_report(flat_references, flat_predictions, labels=label_names)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me

In [ ]:
print(report)

              precision    recall  f1-score   support

           O       0.30      0.94      0.46       154
       B-PER       0.00      0.00      0.00         0
       I-PER       0.00      0.00      0.00         0
       B-LOC       0.00      0.00      0.00         0
       I-LOC       0.00      0.00      0.00         0
       B-ORG       0.00      0.00      0.00         0
       I-ORG       0.00      0.00      0.00         0

   micro avg       0.30      0.94      0.46       154
   macro avg       0.04      0.13      0.07       154
weighted avg       0.30      0.94      0.46       154



In [ ]:
all_predictions

[array([1, 2, 4, 6, 4, 5, 4, 4, 4, 5, 1, 6, 0, 4, 2, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4,
        4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4]),
 array([1, 6, 6, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3,
        4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 4, 4, 4]),
 array([1, 0, 5, 0, 3, 2, 4, 6, 0, 4, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [ ]:
all_labels

[array([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100]),
 array([-100,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100

In [ ]:
true_predictions[0]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
true_labels[0]

['PER_I',
 'LOC_I',
 'ORG_I',
 'LOC_I',
 'ORG_B',
 'LOC_I',
 'LOC_I',
 'LOC_I',
 'ORG_B',
 'PER_B',
 'ORG_I',
 'O',
 'LOC_I']

In [ ]:
def postprocess(predictions, labels):
    # predictions = predictions.detach().cpu().clone().numpy()
    # labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

# Training and Evaluation
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in tqdm(train_dataloader):


        optimizer.zero_grad()

        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss


        loss.backward()
        optimizer.step()
        scheduler.step()


    # Evaluation
    model.eval()
    all_predictions = []
    all_labels = []

    for batch in tqdm(eval_dataloader):
        # Move batch data to device

        batch = {key: value.to(device) for key, value in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Collect predictions and labels for metric calculation
        all_predictions.extend(predictions.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())

    # Compute metrics
    true_predictions, true_labels = postprocess(all_predictions, all_labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)
    results = metric.compute(zero_division=0)



    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )



100%|██████████| 3/3 [00:00<00:00, 13.34it/s]

epoch 0: {'precision': 0.047619047619047616, 'recall': 0.006711409395973154, 'f1': 0.011764705882352943, 'accuracy': 0.2766497461928934}



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG_I seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER_B seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.

In [ ]:
print(true_predictions[5])

['O', 'O', 'O', 'O', 'O', 'O', 'ORG_B', 'ORG_I', 'ORG_I', 'ORG_I', 'O', 'PER_B', 'PER_I', 'PER_I', 'PER_I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER_B', 'PER_I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(true_labels[5])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER_B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
true_labels

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'PER_B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ORG_B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'PER_B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Define the entity labels
entity_labels = {
    "PER": ['PER_B', 'PER_I'],
    "LOC": ['LOC_B', 'LOC_I'],
    "ORG": ['ORG_B', 'ORG_I']
}

# Flatten the true and predicted labels
true_predictions = [['O', 'O', 'O', 'O', 'O', 'O', 'ORG_B', 'ORG_I', 'ORG_I', 'ORG_I', 'O', 'PER_B', 'PER_I', 'PER_I', 'PER_I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER_B', 'PER_I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]  # Example
true_labels = [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER_B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]  # Example

# Flatten predictions and labels
flat_predictions = [item for sublist in true_predictions for item in sublist]
flat_labels = [item for sublist in true_labels for item in sublist]

# Initialize counters for TP, FP, FN for each entity
TP, FP, FN = {'PER': 0, 'LOC': 0, 'ORG': 0}, {'PER': 0, 'LOC': 0, 'ORG': 0}, {'PER': 0, 'LOC': 0, 'ORG': 0}

# Aggregate counts for each entity type
for entity, labels in entity_labels.items():
    for i, label in enumerate(flat_labels):
        # If the true label is part of the current entity
        if label in labels:
            if flat_predictions[i] in labels:
                TP[entity] += 1  # Correct prediction
            else:
                FN[entity] += 1  # Missed prediction
        elif flat_predictions[i] in labels:
            FP[entity] += 1  # False positive

# Compute precision, recall, and F1 score for each entity
precision = {entity: TP[entity] / (TP[entity] + FP[entity]) if (TP[entity] + FP[entity]) > 0 else 0 for entity in entity_labels}
recall = {entity: TP[entity] / (TP[entity] + FN[entity]) if (TP[entity] + FN[entity]) > 0 else 0 for entity in entity_labels}
f1 = {entity: 2 * (precision[entity] * recall[entity]) / (precision[entity] + recall[entity]) if (precision[entity] + recall[entity]) > 0 else 0 for entity in entity_labels}

# Print the results
for entity in entity_labels:
    print(f"Entity {entity} - Precision: {precision[entity]:.4f}, Recall: {recall[entity]:.4f}, F1 Score: {f1[entity]:.4f}")

Entity PER - Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Entity LOC - Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Entity ORG - Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


In [ ]:
all_labels

[array([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100]),
 array([-100,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100

In [ ]:
predictions

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [ ]:
outputs.logits.shape

torch.Size([4, 60, 7])